In [11]:
import gc
import json
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm_notebook
from glob import glob

from sklearn.decomposition import TruncatedSVD
from utils import loadpkl

%matplotlib inline

In [2]:
# load feathers
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm_notebook(files, mininterval=60)], axis=1)

# split train & test
train_df = df[df['click_mode'].notnull()]
test_df = df[df['click_mode'].isnull()]

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [18]:
# load pkls
df = loadpkl('../features/plans.pkl')
queries = loadpkl('../features/queries.pkl')
profiles = loadpkl('../features/profiles.pkl')

In [12]:
# merge
df = pd.merge(df, queries, on='sid', how='left')
df = pd.merge(df, profiles, on='pid', how='left')

del queries, profiles
gc.collect()

train_df = df[df['click_mode'].notnull()]

In [16]:
feats_prof = [f for f in profiles.columns.to_list() if f not in ['pid']]
svd = TruncatedSVD(n_components=20, n_iter=20, random_state=2019)
svd.fit_transform(profiles[feats_prof].values)

array([[ 4.23600254e+03,  5.40932505e+00, -5.80920901e-01, ...,
         2.42769154e-02, -9.44008875e-02,  1.95654909e-03],
       [ 1.48920021e+04,  2.08473709e+00, -1.27411385e+00, ...,
        -2.64398589e-01, -9.53964372e-02, -4.57111015e-02],
       [ 1.48920021e+04,  2.15611720e+00, -1.18560199e+00, ...,
        -1.47757115e-01, -5.98817101e-02, -1.03755162e-01],
       ...,
       [ 2.75870014e+04, -2.05530430e+00,  7.98401476e-01, ...,
         9.05569589e-01, -6.20973589e-02, -1.27454748e-01],
       [ 1.48920021e+04,  2.13455634e+00, -1.07937522e+00, ...,
        -1.71953580e-01, -6.45253441e-02,  1.19827887e-03],
       [ 1.49050029e+04,  4.17349192e+00,  1.00635911e+00, ...,
        -1.04728279e-01, -8.55736671e-02, -1.10135317e-01]])

In [23]:
cols_distance = ['plans_{}_distance'.format(i) for i in range(0,7)]
df[cols_distance].idxmax(axis=1)

0         plans_1_distance
1         plans_0_distance
2         plans_0_distance
3         plans_4_distance
4         plans_1_distance
5         plans_0_distance
6         plans_0_distance
7         plans_3_distance
8         plans_5_distance
9         plans_1_distance
10        plans_4_distance
11        plans_2_distance
12        plans_0_distance
13        plans_4_distance
14        plans_1_distance
15        plans_1_distance
16        plans_1_distance
17        plans_3_distance
18        plans_5_distance
19        plans_3_distance
20        plans_0_distance
21        plans_4_distance
22        plans_0_distance
23        plans_3_distance
24        plans_1_distance
25        plans_2_distance
26        plans_3_distance
27        plans_5_distance
28        plans_4_distance
29        plans_0_distance
                ...       
583595    plans_2_distance
583596    plans_1_distance
583597    plans_4_distance
583598    plans_0_distance
583599    plans_0_distance
583600    plans_0_distance
5

In [22]:
train_df.plans_0_transport_mode.map(train_df.groupby('plans_0_transport_mode').mean()['target_0'])

0         0.060405
1         0.046703
2         0.067123
3         0.046703
4         0.067123
5         0.127590
6         0.046703
7         0.046703
8         0.046703
9         0.092444
10        0.046703
11        0.046703
12        0.067123
13        0.091560
14        0.092444
15        0.067123
16        0.046703
17        0.127590
18        0.156717
19        0.046703
20        0.067123
21        0.046703
22        0.067123
23        0.060405
24        0.046703
25        0.091560
26        0.091560
27        0.091560
28        0.046703
29        0.046703
            ...   
491024    0.060405
491025    0.046703
491026    0.046703
491027    0.060405
491028    0.067123
491029    0.046703
491030    0.067123
491031    0.060405
491032    0.046703
491033    0.091560
491034    0.092444
491035    0.122996
491036    0.067123
491037    0.092444
491038    0.067123
491039    0.067123
491040    0.092444
491041    0.046703
491042    0.060405
491043    0.046703
491044    0.127590
491045    0.

In [23]:
target_dummies.columns.to_list()

['target_0',
 'target_1',
 'target_2',
 'target_3',
 'target_4',
 'target_5',
 'target_6',
 'target_7',
 'target_8',
 'target_9',
 'target_10',
 'target_11']